## Data Loading..

In [48]:
import math

import matplotlib.pyplot as plt

import keras

import pandas as pd

import numpy as np

from keras.models import Sequential

from keras.layers import Dense

from keras.layers import LSTM

from keras.layers import Dropout

from keras.layers import *

from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import mean_squared_error

from sklearn.metrics import mean_absolute_error

from sklearn.model_selection import train_test_split

from keras.callbacks import EarlyStopping

import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import random

import torch.optim as optim

In [49]:
data = pd.read_csv('./Data/MinMaxScaled_Winsorized_Feature_20231027.csv')
# Drop the specified columns from the dataset
data = data.drop(columns=[
    'quote_av',
    'tb_base_av',
    'tb_quote_av',
])

data.head()


,Open_time,Open,High,Low,Close,Volume,trades,VIX,Bitcoin_TransactionFee,volatility,GoogleTrend,DowJones,EuroStoxx50,Nasdaq,OilWTI,SPX,TradePermin,Dominance,Gold
0,2018-01-03,0.926926,0.442124,0.908569,0.569227,0.042299,0.019350,0.000000,1.0,1.000000,0.586219,0.547234,0.571035,0.582848,0.533921,0.576066,0.535315,0.130527,0.535216
1,2018-01-04,0.564277,0.486714,0.425847,0.537085,0.058249,0.020318,0.001940,1.0,1.000000,0.574495,0.573304,0.714814,0.514008,0.514174,0.547026,0.590266,0.130527,0.489068
2,2018-01-05,0.536732,1.000000,0.702397,1.000000,0.063900,0.023233,0.001940,1.0,1.000000,0.551046,0.605995,0.638188,0.582004,0.493552,0.583846,0.621124,0.130527,0.532305
3,2018-01-06,1.000000,0.486078,0.903958,0.524104,0.050138,0.018765,0.010254,1.0,0.907375,0.515873,0.492669,0.529880,0.526148,0.512233,0.518033,0.419330,0.130527,0.532305
4,2018-01-07,0.523761,0.483127,0.389201,0.257298,0.032264,0.013878,0.010254,1.0,0.911113,0.480700,0.492669,0.529880,0.526148,0.512233,0.518033,0.426805,0.072534,0.532305


In [50]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2067 entries, 0 to 2066
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Open_time               2067 non-null   object 
 1   Open                    2067 non-null   float64
 2   High                    2067 non-null   float64
 3   Low                     2067 non-null   float64
 4   Close                   2067 non-null   float64
 5   Volume                  2067 non-null   float64
 6   trades                  2067 non-null   float64
 7   VIX                     2067 non-null   float64
 8   Bitcoin_TransactionFee  2067 non-null   float64
 9   volatility              2067 non-null   float64
 10  GoogleTrend             2067 non-null   float64
 11  DowJones                2067 non-null   float64
 12  EuroStoxx50             2067 non-null   float64
 13  Nasdaq                  2067 non-null   float64
 14  OilWTI                  2067 non-null   

In [51]:
original_close = data['Close'].values
dates = pd.to_datetime(data['Open_time'])

In [52]:
# variables for training
cols = list(data)[1:19]
data_for_train = data[cols].astype(float)

data_for_train


,Open,High,Low,Close,Volume,trades,VIX,Bitcoin_TransactionFee,volatility,GoogleTrend,DowJones,EuroStoxx50,Nasdaq,OilWTI,SPX,TradePermin,Dominance,Gold
0,0.926926,0.442124,0.908569,0.569227,0.042299,0.019350,0.000000,1.000000,1.000000,0.586219,0.547234,0.571035,0.582848,0.533921,0.576066,0.535315,0.130527,0.535216
1,0.564277,0.486714,0.425847,0.537085,0.058249,0.020318,0.001940,1.000000,1.000000,0.574495,0.573304,0.714814,0.514008,0.514174,0.547026,0.590266,0.130527,0.489068
2,0.536732,1.000000,0.702397,1.000000,0.063900,0.023233,0.001940,1.000000,1.000000,0.551046,0.605995,0.638188,0.582004,0.493552,0.583846,0.621124,0.130527,0.532305
3,1.000000,0.486078,0.903958,0.524104,0.050138,0.018765,0.010254,1.000000,0.907375,0.515873,0.492669,0.529880,0.526148,0.512233,0.518033,0.419330,0.130527,0.532305
4,0.523761,0.483127,0.389201,0.257298,0.032264,0.013878,0.010254,1.000000,0.911113,0.480700,0.492669,0.529880,0.526148,0.512233,0.518033,0.426805,0.072534,0.532305
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2062,0.488047,0.506510,0.491951,0.509925,0.031107,0.043323,0.164349,0.029851,0.100026,0.468975,0.574360,0.672967,0.583431,0.510437,0.574422,0.000702,0.398208,0.479670
2063,0.509599,0.509407,0.478626,0.498667,0.062257,0.065709,0.164349,0.042706,0.099933,0.527597,0.574360,0.672967,0.583431,0.510437,0.574422,0.020247,0.398208,0.479670
2064,0.498354,0.855231,0.504577,0.765847,0.213876,0.144722,0.146888,0.058046,0.156660,0.691739,0.601978,0.596692,0.676966,0.523665,0.675428,0.077595,0.398208,0.635801
2065,0.765207,0.429252,0.673863,0.429149,0.099767,0.087229,0.131091,0.050959,0.158820,0.586219,0.512095,0.465197,0.552157,0.513671,0.544631,0.033075,0.398208,0.664190


In [53]:
# split to train data and test data
n_train = int(0.95*data_for_train.shape[0])
train_data = data_for_train[0: n_train]
train_dates = dates[0: n_train]

test_data = data_for_train[n_train:]
test_dates = dates[n_train:]

train_data.shape, test_data.shape # 2D Arrays , (1860,18) , (207,18)

((1963, 18), (104, 18))

In [54]:
class windowDataset(Dataset):
    def __init__(self, data, target, input_window, output_window, stride=1):
        data = data.values 
        L, features = data.shape
        num_samples = (L - input_window - output_window) // stride + 1

        X = np.zeros([input_window, num_samples, features])
        Y = np.zeros([output_window, num_samples, 1])  # 'Close' 열만을 대상으로 하므로 1로 설정

        for i in np.arange(num_samples):
            start_x = stride*i
            end_x = start_x + input_window
            X[:,i,:] = data[start_x:end_x, :]

            start_y = stride*i + input_window
            end_y = start_y + output_window
            Y[:,i,0] = target[start_y:end_y]  # 'Close' 열만을 대상으로 합니다.

        self.x = X.transpose((1,0,2))
        self.y = Y.transpose((1,0,2))
        self.len = len(X)

    def __getitem__(self, i):
        return self.x[i], self.y[i]

    def __len__(self):
        return self.len


In [55]:
class lstm_encoder(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=1): 
        super(lstm_encoder, self).__init__()
        
        # LSTM Layer 초기화
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)

    # 모델의 순전파함수, 모델이 학습 및 예측 수행할 때 호출
    def forward(self, x_input): # x_input : 모델의 입력 데이터
        lstm_out, hidden = self.lstm(x_input)
        return lstm_out, hidden


In [56]:
class lstm_decoder(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=1):
        super(lstm_decoder, self).__init__()
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, input_size)

    def forward(self, x_input, encoder_hidden_states):
        lstm_out, hidden = self.lstm(x_input, encoder_hidden_states)
        output = self.linear(lstm_out)
        return output, hidden

In [57]:
class lstm_encoder_decoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(lstm_encoder_decoder, self).__init__()
        self.encoder = lstm_encoder(input_size=input_size, hidden_size=hidden_size)
        self.decoder = lstm_decoder(input_size=input_size, hidden_size=hidden_size)

    def forward(self, inputs, targets, target_len, teacher_forcing_ratio):
        batch_size, _, input_size = inputs.shape
        outputs = torch.zeros(batch_size, target_len, input_size)

        _, hidden = self.encoder(inputs)
        decoder_input = inputs[:,-1, :]

        for t in range(target_len): 
            out, hidden = self.decoder(decoder_input.unsqueeze(1), hidden)
            out = out.squeeze(1)
            if random.random() < teacher_forcing_ratio:
                decoder_input = targets[:, t, :]
            else:
                decoder_input = out
            outputs[:,t,:] = out

        return outputs

    def predict(self, inputs, target_len):
        self.eval()
        batch_size, _, input_size = inputs.shape
        outputs = torch.zeros(batch_size, target_len, input_size)
        _, hidden = self.encoder(inputs)
        decoder_input = inputs[:,-1, :]
        for t in range(target_len): 
            out, hidden = self.decoder(decoder_input.unsqueeze(1), hidden)
            out = out.squeeze(1)
            decoder_input = out
            outputs[:,t,:] = out
        return outputs.detach().numpy()

In [58]:
iw = 312
ow = 104

target_data = train_data['Close'].values  # 'Close' 열만을 대상으로 합니다.

train_dataset = windowDataset(train_data, target_data, input_window=iw, output_window=ow, stride=1)
train_loader = DataLoader(train_dataset, batch_size=64)

In [59]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [60]:
model = lstm_encoder_decoder(input_size=18, hidden_size=16).to(device)

In [61]:
learning_rate=0.01
epoch = 3000
optimizer = optim.Adam(model.parameters(), lr = learning_rate)
criterion = nn.MSELoss()

In [62]:
from tqdm import tqdm

model.train()
with tqdm(range(epoch)) as tr:
    for i in tr:
        total_loss = 0.0
        for x,y in train_loader:
            optimizer.zero_grad()
            x = x.to(device).float()
            y = y.to(device).float()
            output = model(x, y, ow, 0.6).to(device)
            loss = criterion(output, y)
            loss.backward()
            optimizer.step()
            total_loss += loss.cpu().item()
        tr.set_postfix(loss="{0:.5f}".format(total_loss/len(train_loader)))

  0%|          | 0/3000 [00:00<?, ?it/s]


RuntimeError: input.size(-1) must be equal to input_size. Expected 18, got 1